In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast
from datetime import datetime, timedelta
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.preprocessing import StandardScaler
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import intercluster_distance
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from wordcloud import WordCloud

### (1) 데이터 불러오기

In [2]:
df = pd.read_csv("/Users/jun/GitStudy/Data_4/src/project/project5/tiktok/0130/tiktoker_crawling_df_0127.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1680 non-null   int64 
 1   name        1680 non-null   object
 2   follower    1680 non-null   object
 3   view        1680 non-null   object
 4   like        1680 non-null   object
 5   comment     1680 non-null   int64 
 6   save        1680 non-null   object
 7   date        1680 non-null   object
 8   info_tag    1664 non-null   object
 9   hash_tag    1680 non-null   object
dtypes: int64(2), object(8)
memory usage: 131.4+ KB


### (2)전처리

#### 전처리_5 : 숫자형, 날짜형 컬럼
> - view, like, comment, save 컬럼 : 단위(k,m) 변환
> - date
    >> - 25년도 1월 업로드한 경우 2025-01-nn으로 변환 
    >> - nh ago, nd ago, 1w ago인 경우 2025-01-21기준으로 날짜 계산

##### 5-1. date 컬럼 형식 통일 및 datetime으로 변환

In [4]:
def preprocess_date(df):
    today = datetime(2025, 1, 21)

    for i in range(len(df)):
    
        if len(df['date'][i]) >= 8:
            df.loc[i, 'upload_date'] = df['date'][i]
    
        elif len(df['date'][i]) >= 6:
            date = df['date'][i].replace(' ','')
            alphabet_list = [a for a in date]
    
            if 'h' in alphabet_list:
                df.loc[i, 'upload_date'] = today
    
            elif 'd' in alphabet_list:
                day = alphabet_list[0]
                day = int(day)
                df.loc[i, 'upload_date'] = today - timedelta(days=day)
    
            elif 'w' in alphabet_list:
                week = alphabet_list[0]
                week = int(week)
                df.loc[i, 'upload_date'] = today - timedelta(weeks=week)
    
        elif len(df['date'][i]) == 4:
            df.loc[i, 'upload_date'] = '2025-' + df['date'][i]
    
        elif len(df['date'][i]) == 3:
            df.loc[i, 'upload_date'] = '2025-' + df['date'][i]

    # datetime형으로 변환
    df['upload_date'] = pd.to_datetime(df['upload_date'], format="%Y-%m-%d", errors='coerce')


preprocess_date(df)

In [5]:
# 결과 확인 1
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   1680 non-null   int64         
 1   name         1680 non-null   object        
 2   follower     1680 non-null   object        
 3   view         1680 non-null   object        
 4   like         1680 non-null   object        
 5   comment      1680 non-null   int64         
 6   save         1680 non-null   object        
 7   date         1680 non-null   object        
 8   info_tag     1664 non-null   object        
 9   hash_tag     1680 non-null   object        
 10  upload_date  1680 non-null   datetime64[us]
dtypes: datetime64[us](1), int64(2), object(8)
memory usage: 144.5+ KB


#### view, like, comment, save컬럼 단위 변환

In [6]:
# follower 컬럼 단위 전처리

def process_follower(df):
    for i in range(len(df)):
        follower = df['follower'][i].replace(' ','')
        follower_list = [a for a in follower]
        if 'K' in follower_list:
            num = follower.split('K')[0]
            num = float(num)
            follower_cal = num*1000
            df.loc[i, 'follower_cnt'] = follower_cal
        elif 'M' in follower_list:
            num = follower.split('M')[0]
            num = float(num)
            follower_cal = num*1000000
            df.loc[i, 'follower_cnt'] = follower_cal
        else:
            num = df['follower'][i].replace(' ','')
            follower_cal = float(num)
            df.loc[i, 'follower_cnt'] = follower_cal

process_follower(df)

In [7]:
# view 컬럼 단위 전처리

def process_view(df):
    for i in range(len(df)):
        view = df['view'][i].replace(' ','')
        view_list = [a for a in view]
        if 'K' in view_list:
            num = view.split('K')[0]
            num = float(num)
            view_cal = num*1000
            df.loc[i, 'view_cnt'] = view_cal
        elif 'M' in view_list:
            num = view.split('M')[0]
            num = float(num)
            view_cal = num*1000000
            df.loc[i, 'view_cnt'] = view_cal
        else:
            num = df['view'][i].replace(' ','')
            view_cal = float(num)
            df.loc[i, 'view_cnt'] = view_cal

process_view(df)

In [8]:
# like 컬럼 단위 전처리

def process_like(df):
    for i in range(len(df)):
        like = df['like'][i].replace(' ','')
        like_list = [a for a in like]
        if 'K' in like_list:
            num = like.split('K')[0]
            num = float(num)
            like_cal = num*1000
            df.loc[i, 'like_cnt'] = like_cal
        elif 'M' in like_list:
            num = like.split('M')[0]
            num = float(num)
            like_cal = num*1000000
            df.loc[i, 'like_cnt'] = like_cal
        else:
            num = df['like'][i].replace(' ','')
            like_cal = float(num)
            df.loc[i, 'like_cnt'] = like_cal

process_like(df)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    1680 non-null   int64         
 1   name          1680 non-null   object        
 2   follower      1680 non-null   object        
 3   view          1680 non-null   object        
 4   like          1680 non-null   object        
 5   comment       1680 non-null   int64         
 6   save          1680 non-null   object        
 7   date          1680 non-null   object        
 8   info_tag      1664 non-null   object        
 9   hash_tag      1680 non-null   object        
 10  upload_date   1680 non-null   datetime64[us]
 11  follower_cnt  1680 non-null   float64       
 12  view_cnt      1680 non-null   float64       
 13  like_cnt      1680 non-null   float64       
dtypes: datetime64[us](1), float64(3), int64(2), object(8)
memory usage: 183.9+ KB


In [10]:
# comment 컬럼 단위 전처리

def process_comment(df):
    for i in range(len(df)):
        comment = str(df['comment'][i]).replace(' ','')
        comment_list = [a for a in comment]
        if 'K' in comment_list:
            num = comment.split('K')[0]
            num = float(num)
            comment_cal = num*1000
            df.loc[i, 'comment_cnt'] = comment_cal
        elif 'M' in comment_list:
            num = comment.split('M')[0]
            num = float(num)
            comment_cal = num*1000000
            df.loc[i, 'comment_cnt'] = comment_cal
        else:
            num = str(df['comment'][i]).replace(' ','')
            comment_cal = float(num)
            df.loc[i, 'comment_cnt'] = comment_cal

process_comment(df)

In [11]:
# save 컬럼 단위 전처리

def process_save(df):
    for i in range(len(df)):
        save = df['save'][i].replace(' ','')
        save_list = [a for a in save]
        if 'K' in save_list:
            num = save.split('K')[0]
            num = float(num)
            save_cal = num*1000
            df.loc[i, 'save_cnt'] = save_cal
        elif 'M' in save_list:
            num = save.split('M')[0]
            num = float(num)
            save_cal = num*1000000
            df.loc[i, 'save_cnt'] = save_cal
        else:
            num = df['save'][i].replace(' ','')
            save_cal = float(num)
            df.loc[i, 'save_cnt'] = save_cal

process_save(df)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    1680 non-null   int64         
 1   name          1680 non-null   object        
 2   follower      1680 non-null   object        
 3   view          1680 non-null   object        
 4   like          1680 non-null   object        
 5   comment       1680 non-null   int64         
 6   save          1680 non-null   object        
 7   date          1680 non-null   object        
 8   info_tag      1664 non-null   object        
 9   hash_tag      1680 non-null   object        
 10  upload_date   1680 non-null   datetime64[us]
 11  follower_cnt  1680 non-null   float64       
 12  view_cnt      1680 non-null   float64       
 13  like_cnt      1680 non-null   float64       
 14  comment_cnt   1680 non-null   float64       
 15  save_cnt      1680 non-null   float64 

In [13]:
# view_cnt, like_cnt, comment_cnt, save_cnt컬럼 float에서 int로 변환

df['view_cnt'] = df['view_cnt'].round(0).astype(int)
df['like_cnt'] = df['like_cnt'].round(0).astype(int)
df['comment_cnt'] = df['comment_cnt'].round(0).astype(int)
df['save_cnt'] = df['save_cnt'].round(0).astype(int)

In [14]:
# 결과 확인 1
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    1680 non-null   int64         
 1   name          1680 non-null   object        
 2   follower      1680 non-null   object        
 3   view          1680 non-null   object        
 4   like          1680 non-null   object        
 5   comment       1680 non-null   int64         
 6   save          1680 non-null   object        
 7   date          1680 non-null   object        
 8   info_tag      1664 non-null   object        
 9   hash_tag      1680 non-null   object        
 10  upload_date   1680 non-null   datetime64[us]
 11  follower_cnt  1680 non-null   float64       
 12  view_cnt      1680 non-null   int64         
 13  like_cnt      1680 non-null   int64         
 14  comment_cnt   1680 non-null   int64         
 15  save_cnt      1680 non-null   int64   

In [15]:
# 결과 확인 2
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Unnamed: 0    1680 non-null   int64         
 1   name          1680 non-null   object        
 2   follower      1680 non-null   object        
 3   view          1680 non-null   object        
 4   like          1680 non-null   object        
 5   comment       1680 non-null   int64         
 6   save          1680 non-null   object        
 7   date          1680 non-null   object        
 8   info_tag      1664 non-null   object        
 9   hash_tag      1680 non-null   object        
 10  upload_date   1680 non-null   datetime64[us]
 11  follower_cnt  1680 non-null   float64       
 12  view_cnt      1680 non-null   int64         
 13  like_cnt      1680 non-null   int64         
 14  comment_cnt   1680 non-null   int64         
 15  save_cnt      1680 non-null   int64   

In [16]:
df.columns

Index(['Unnamed: 0', 'name', 'follower', 'view', 'like', 'comment', 'save',
       'date', 'info_tag', 'hash_tag', 'upload_date', 'follower_cnt',
       'view_cnt', 'like_cnt', 'comment_cnt', 'save_cnt'],
      dtype='object')

In [17]:
# 필요한 컬럼만 추출

df = df[['name', 'follower_cnt', 'upload_date', 'view_cnt', 'like_cnt','comment_cnt', 'save_cnt', 'info_tag']]

In [18]:
df

,name,follower_cnt,upload_date,view_cnt,like_cnt,comment_cnt,save_cnt,info_tag
0,mydelicate,399800.0,2025-01-21,5158,1334,74,175,my favourite blushes and why (with shades)💓 yo...
1,mydelicate,399800.0,2025-01-18,24300,6606,246,917,spend a productive morning with me 💓🎀 why is i...
2,mydelicate,399800.0,2025-01-15,10400,1881,90,218,for my girlies who still want to look cute whe...
3,mydelicate,399800.0,2025-01-14,22100,4206,193,602,new Flower Knows strawberry cupid collection u...
4,mydelicate,399800.0,2025-01-18,17200,2995,123,432,my favourite products for soft skin 🎀💓 names:\...
...,...,...,...,...,...,...,...,...
1675,amyflamy1,4300000.0,2024-12-14,1400000,131400,640,3686,My hands be too smoll\n#fyp #dance #dancechall...
1676,amyflamy1,4300000.0,2024-12-13,3800000,431000,1654,21100,We single for a reason✨💅\n\n#fyp #dance #dance...
1677,amyflamy1,4300000.0,2024-12-12,6200000,647700,3055,33400,Bibim noodels mukbang🍜\n#mukbang #mukbangeatin...
1678,amyflamy1,4300000.0,2024-12-10,936900,94400,536,3589,Welcome girls you’re safe here\n#gurwm #gurwmr...


### 전처리_2 : 해시태그 컬럼 생성
> - 크롤링 한 데이터를 csv파일로 변환하는 과정에서 hash_tag컬럼의 일부 행들이 손상되는 문제 발생
> - info 컬럼에서 해시태그 추출하여 해시태그 컬럼 다시 생성

#### info 컬럼에서 해시태그 추출

In [19]:
def new_hash_tag_process(df):
    for i in range(len(df)):
        info = str(df['info_tag'][i])
        info_split_list = info.split(' ')
        hash_tag_list = []
        for a in info_split_list:
            if a.startswith('#'):
                hash_tag_list.append(a)
            elif a.startswith('@'):
                hash_tag_list.append(a)
            else:
                pass
        df.loc[i, 'hash_tag'] = ','.join(hash_tag_list)

new_hash_tag_process(df)

In [20]:
# 결과 확인
df.head(3)

,name,follower_cnt,upload_date,view_cnt,like_cnt,comment_cnt,save_cnt,info_tag,hash_tag
0,mydelicate,399800.0,2025-01-21,5158,1334,74,175,my favourite blushes and why (with shades)💓 yo...,"@YesStyle,@House,#cbeauty,#koreanmakeup,#skinc..."
1,mydelicate,399800.0,2025-01-18,24300,6606,246,917,spend a productive morning with me 💓🎀 why is i...,"#kbeauty,#morningroutine,#vlog,#wonyoungism,#s..."
2,mydelicate,399800.0,2025-01-15,10400,1881,90,218,for my girlies who still want to look cute whe...,"@SHEINUS,@SHEIN\nuse,#winterclothes,#coquette,..."


In [21]:
# 결과 확인 2
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   name          1680 non-null   object        
 1   follower_cnt  1680 non-null   float64       
 2   upload_date   1680 non-null   datetime64[us]
 3   view_cnt      1680 non-null   int64         
 4   like_cnt      1680 non-null   int64         
 5   comment_cnt   1680 non-null   int64         
 6   save_cnt      1680 non-null   int64         
 7   info_tag      1664 non-null   object        
 8   hash_tag      1680 non-null   object        
dtypes: datetime64[us](1), float64(1), int64(4), object(3)
memory usage: 118.3+ KB


In [ ]:
data_path = ''
os.chdir(data_path)
df.to_csv('tiktoker_final_df_0127.csv',index=False, encoding="utf-8-sig")

In [23]:
df

,name,follower_cnt,upload_date,view_cnt,like_cnt,comment_cnt,save_cnt,info_tag,hash_tag
0,mydelicate,399800.0,2025-01-21,5158,1334,74,175,my favourite blushes and why (with shades)💓 yo...,"@YesStyle,@House,#cbeauty,#koreanmakeup,#skinc..."
1,mydelicate,399800.0,2025-01-18,24300,6606,246,917,spend a productive morning with me 💓🎀 why is i...,"#kbeauty,#morningroutine,#vlog,#wonyoungism,#s..."
2,mydelicate,399800.0,2025-01-15,10400,1881,90,218,for my girlies who still want to look cute whe...,"@SHEINUS,@SHEIN\nuse,#winterclothes,#coquette,..."
3,mydelicate,399800.0,2025-01-14,22100,4206,193,602,new Flower Knows strawberry cupid collection u...,"#flowerknows,#unboxing,#cbeauty,#chinesemakeup..."
4,mydelicate,399800.0,2025-01-18,17200,2995,123,432,my favourite products for soft skin 🎀💓 names:\...,"@cosmetic_jolse,@YesStyle,@sephora,@Sol,#selfc..."
...,...,...,...,...,...,...,...,...,...
1675,amyflamy1,4300000.0,2024-12-14,1400000,131400,640,3686,My hands be too smoll\n#fyp #dance #dancechall...,"#dance,#dancechallenge"
1676,amyflamy1,4300000.0,2024-12-13,3800000,431000,1654,21100,We single for a reason✨💅\n\n#fyp #dance #dance...,"#dance,#dancechallenge,#dresstoimpress,#fashio..."
1677,amyflamy1,4300000.0,2024-12-12,6200000,647700,3055,33400,Bibim noodels mukbang🍜\n#mukbang #mukbangeatin...,"#mukbangeatingshow,#mukbangvideo,#eating,#eati..."
1678,amyflamy1,4300000.0,2024-12-10,936900,94400,536,3589,Welcome girls you’re safe here\n#gurwm #gurwmr...,"#gurwmroutine,#grwm,#grwmroutine,#skincarerout..."


### 전처리_4 : 자연어 처리

In [24]:
# 데이터 다시 불러오기 
df = pd.read_csv(data_path+'tiktoker_final_df_0127.csv')

In [25]:
df.head(3)

,name,follower_cnt,upload_date,view_cnt,like_cnt,comment_cnt,save_cnt,info_tag,hash_tag
0,mydelicate,399800.0,2025-01-21,5158,1334,74,175,my favourite blushes and why (with shades)💓 yo...,"@YesStyle,@House,#cbeauty,#koreanmakeup,#skinc..."
1,mydelicate,399800.0,2025-01-18,24300,6606,246,917,spend a productive morning with me 💓🎀 why is i...,"#kbeauty,#morningroutine,#vlog,#wonyoungism,#s..."
2,mydelicate,399800.0,2025-01-15,10400,1881,90,218,for my girlies who still want to look cute whe...,"@SHEINUS,@SHEIN\nuse,#winterclothes,#coquette,..."


In [26]:
# nltk 에서 Punkt tokenizer & stopwords list를 다운로드
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /Users/jun/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/jun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### EDA

광고 가격 분류
평균 영상 조회 수 * 2만

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          1680 non-null   object 
 1   follower_cnt  1680 non-null   float64
 2   upload_date   1680 non-null   object 
 3   view_cnt      1680 non-null   int64  
 4   like_cnt      1680 non-null   int64  
 5   comment_cnt   1680 non-null   int64  
 6   save_cnt      1680 non-null   int64  
 7   info_tag      1664 non-null   object 
 8   hash_tag      1620 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 118.3+ KB


#### 인플루언서별 평균 view, like, comment, save

In [28]:
# 소수점 이하 3자리까지 출력되도록 설정
pd.options.display.float_format = '{:,.3f}'.format

mean_df = df.groupby('name').agg({'follower_cnt':'mean',
                                    'view_cnt':'mean',
                                    'like_cnt':'mean',
                                    'comment_cnt':'mean',
                                    'save_cnt':'mean'
                                    }).reset_index()

In [29]:
mean_df

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt
0,.blissdiaries,"81,700.000","9,717.967","1,447.333",60.467,196.200
1,.m.egan,"238,200.000","178,070.000","23,864.300",182.533,"3,527.300"
2,.woniluv,"135,600.000","41,956.200","2,399.000",85.767,369.933
3,4chriisty,"79,500.000","26,122.500","4,119.733",74.133,385.367
4,__.annie.s,"37,300.000","10,332.733","2,375.100",128.467,465.100
5,_luvwonsnnx,"23,900.000","8,783.467","1,593.633",57.867,261.967
6,aeiezo.pdf,"21,400.000","20,647.133","2,587.733",95.900,350.233
7,ameenbeautytips,"4,800,000.000","418,445.967","1,602.867",14.767,143.100
8,amyflamy1,"4,300,000.000","2,490,070.000","272,720.000","1,466.933","19,117.433"
9,asiamelon,"13,700.000","1,751.233",164.300,12.267,19.767


In [30]:
mean_df.loc[:,'ad_cost'] = mean_df['view_cnt'] * 20000

In [31]:
# erp
mean_df['ER%'] = (mean_df['like_cnt']+mean_df['comment_cnt'])/mean_df['follower_cnt'] * 100 

In [32]:
# 팔로워 수 기준 인플루언서 카테고리 분류 함수
def classify_influencer(follower_cnt):
    if follower_cnt < 10_000:
        return "나노 인플루언서"
    elif 10_000 <= follower_cnt < 100_000:
        return "마이크로 인플루언서"
    elif 500_000 <= follower_cnt < 1_000_000:
        return "매크로 인플루언서"
    else:
        return "메가 인플루언서"

# 쉼표 제거 후 int 변환
mean_df["follower_cnt"] = mean_df["follower_cnt"].replace(",", "", regex=True).astype(float).astype(int)

# 카테고리 적용
mean_df["influencer_size"] = mean_df["follower_cnt"].apply(classify_influencer)

# 결과 확인
print(mean_df[["name", "follower_cnt", "influencer_size"]])

                 name  follower_cnt influencer_size
0       .blissdiaries         81700      마이크로 인플루언서
1             .m.egan        238200        메가 인플루언서
2            .woniluv        135600        메가 인플루언서
3           4chriisty         79500      마이크로 인플루언서
4          __.annie.s         37300      마이크로 인플루언서
5         _luvwonsnnx         23900      마이크로 인플루언서
6          aeiezo.pdf         21400      마이크로 인플루언서
7     ameenbeautytips       4800000        메가 인플루언서
8           amyflamy1       4300000        메가 인플루언서
9           asiamelon         13700      마이크로 인플루언서
10           ayheyt3u         78900      마이크로 인플루언서
11         aylennpark       1400000        메가 인플루언서
12         bwnniewony        232600        메가 인플루언서
13   caspertheghostyy        442700        메가 인플루언서
14     cleangirlhacks        153200        메가 인플루언서
15   cleanlivingkarly         25000      마이크로 인플루언서
16         danicolexx        881700       매크로 인플루언서
17         dearwonii_        217100        메가 인플루언서
18          

In [33]:
mean_df.head(5)

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,ad_cost,ER%,influencer_size
0,.blissdiaries,81700,"9,717.967","1,447.333",60.467,196.200,"194,359,333.333",1.846,마이크로 인플루언서
1,.m.egan,238200,"178,070.000","23,864.300",182.533,"3,527.300","3,561,400,000.000",10.095,메가 인플루언서
2,.woniluv,135600,"41,956.200","2,399.000",85.767,369.933,"839,124,000.000",1.832,메가 인플루언서
3,4chriisty,79500,"26,122.500","4,119.733",74.133,385.367,"522,450,000.000",5.275,마이크로 인플루언서
4,__.annie.s,37300,"10,332.733","2,375.100",128.467,465.100,"206,654,666.667",6.712,마이크로 인플루언서


In [34]:
# 업로드 빈도
df[['name','upload_date']]

,name,upload_date
0,mydelicate,2025-01-21
1,mydelicate,2025-01-18
2,mydelicate,2025-01-15
3,mydelicate,2025-01-14
4,mydelicate,2025-01-18
...,...,...
1675,amyflamy1,2024-12-14
1676,amyflamy1,2024-12-13
1677,amyflamy1,2024-12-12
1678,amyflamy1,2024-12-10


In [35]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

# ✅ 현재 날짜 기준
TODAY = datetime(2025, 1, 27)  # 현재 날짜 설정

df['upload_date'] = pd.to_datetime(df['upload_date'])  # 날짜형으로 변환

# ✅ name 별로 업로드 간격 계산
tiktoker_df = df.sort_values(by=["name", "upload_date"])  # 이름 + 날짜순 정렬
tiktoker_df["prev_date"] = tiktoker_df.groupby("name")["upload_date"].shift(1)  # 이전 영상 날짜
tiktoker_df["upload_gap"] = (tiktoker_df["upload_date"] - tiktoker_df["prev_date"]).dt.days  # 간격(일)

# ✅ 평균 업로드 주기 계산
upload_cycle = tiktoker_df.groupby("name")["upload_gap"].mean().reset_index()
upload_cycle.rename(columns={"upload_gap": "avg_upload_interval"}, inplace=True)

# ✅ 결과 출력
print(upload_cycle)

                 name  avg_upload_interval
0       .blissdiaries                4.069
1             .m.egan                5.034
2            .woniluv                2.379
3           4chriisty                2.483
4          __.annie.s                3.138
5         _luvwonsnnx                1.483
6          aeiezo.pdf                3.483
7     ameenbeautytips                0.172
8           amyflamy1                1.448
9           asiamelon                3.345
10           ayheyt3u                5.759
11         aylennpark                1.621
12         bwnniewony                1.931
13   caspertheghostyy                0.828
14     cleangirlhacks                3.034
15   cleanlivingkarly                0.552
16         danicolexx                0.310
17         dearwonii_                2.655
18             emchu_                3.414
19          fromh1ae_                2.207
20            gae4hrv                3.586
21     glowupwithinah                0.241
22         

In [36]:
upload_cycle

,name,avg_upload_interval
0,.blissdiaries,4.069
1,.m.egan,5.034
2,.woniluv,2.379
3,4chriisty,2.483
4,__.annie.s,3.138
5,_luvwonsnnx,1.483
6,aeiezo.pdf,3.483
7,ameenbeautytips,0.172
8,amyflamy1,1.448
9,asiamelon,3.345


In [37]:
mean_df['avg_upload_interval'] = upload_cycle['avg_upload_interval']
mean_df.head(4)

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,ad_cost,ER%,influencer_size,avg_upload_interval
0,.blissdiaries,81700,"9,717.967","1,447.333",60.467,196.200,"194,359,333.333",1.846,마이크로 인플루언서,4.069
1,.m.egan,238200,"178,070.000","23,864.300",182.533,"3,527.300","3,561,400,000.000",10.095,메가 인플루언서,5.034
2,.woniluv,135600,"41,956.200","2,399.000",85.767,369.933,"839,124,000.000",1.832,메가 인플루언서,2.379
3,4chriisty,79500,"26,122.500","4,119.733",74.133,385.367,"522,450,000.000",5.275,마이크로 인플루언서,2.483


In [38]:
#top5_hash_tag_df = pd.read_csv('/Users/jun/GitStudy/Data_4/src/project/project5/tiktok/0130/tiktoker_top5_hashtags.csv')
# 이거 대신 토픽모델링으로 만든 태그 사용

In [39]:
#/Users/jun/GitStudy/Data_4/src/project/project5/tiktok/0130/tiktoker_lableing.ipynb 에서 옴
top3_tag_df = pd.read_csv('tiktoker_top3_modeled_topic.csv')
top3_tag_df.head(3)

,name,color_tf,skincare_tf,hair_body_tf,fashiion_tf,unboxing_tf,skin_routine_tf,trend_tf,asmr,eating_tf,others_tf,no.1,no.2,no.3
0,.blissdiaries,18,17,1,29,17,5,3,8,1,0,fashiion_tf,color_tf,skincare_tf
1,.m.egan,5,13,1,3,4,5,0,2,2,5,skincare_tf,color_tf,skin_routine_tf
2,.woniluv,6,9,2,3,4,9,7,7,1,4,skincare_tf,skin_routine_tf,trend_tf


In [51]:
top3_tag_df['no.1'].value_counts()

no.1
skincare_tf        35
asmr                8
fashiion_tf         3
others_tf           3
color_tf            2
trend_tf            2
hair_body_tf        1
eating_tf           1
skin_routine_tf     1
Name: count, dtype: int64

In [40]:
tags_df = top3_tag_df[['name','no.1','no.2','no.3']]
tags_df.head(3)

,name,no.1,no.2,no.3
0,.blissdiaries,fashiion_tf,color_tf,skincare_tf
1,.m.egan,skincare_tf,color_tf,skin_routine_tf
2,.woniluv,skincare_tf,skin_routine_tf,trend_tf


In [41]:
mean_df = mean_df.merge(tags_df, on='name', how='left')
mean_df.head(3)

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,ad_cost,ER%,influencer_size,avg_upload_interval,no.1,no.2,no.3
0,.blissdiaries,81700,"9,717.967","1,447.333",60.467,196.200,"194,359,333.333",1.846,마이크로 인플루언서,4.069,fashiion_tf,color_tf,skincare_tf
1,.m.egan,238200,"178,070.000","23,864.300",182.533,"3,527.300","3,561,400,000.000",10.095,메가 인플루언서,5.034,skincare_tf,color_tf,skin_routine_tf
2,.woniluv,135600,"41,956.200","2,399.000",85.767,369.933,"839,124,000.000",1.832,메가 인플루언서,2.379,skincare_tf,skin_routine_tf,trend_tf


In [42]:
# 쉼표 제거 및 숫자로 변환
cols_to_int = ["follower_cnt", "view_cnt", "like_cnt", "comment_cnt", "save_cnt", "ad_cost"]
cols_to_float = ["ER%", "avg_upload_interval"]

# int로 변환할 컬럼
mean_df[cols_to_int] = mean_df[cols_to_int].replace(",", "", regex=True).astype(float).round(0).astype(int)

# float (소수 둘째 자리까지)로 변환할 컬럼
mean_df[cols_to_float] = mean_df[cols_to_float].replace(",", "", regex=True).astype(float).round(2)
mean_df.head(3)

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,ad_cost,ER%,influencer_size,avg_upload_interval,no.1,no.2,no.3
0,.blissdiaries,81700,9718,1447,60,196,194359333,1.850,마이크로 인플루언서,4.070,fashiion_tf,color_tf,skincare_tf
1,.m.egan,238200,178070,23864,183,3527,3561400000,10.100,메가 인플루언서,5.030,skincare_tf,color_tf,skin_routine_tf
2,.woniluv,135600,41956,2399,86,370,839124000,1.830,메가 인플루언서,2.380,skincare_tf,skin_routine_tf,trend_tf


In [43]:
os.getcwd()

'/Users/jun/GitStudy/Data_4/src/project/project5/tiktok/recommend/data'

In [44]:
mean_df.to_csv('merged_mean_0207.csv',index=False, encoding="utf-8")

In [45]:
mean_df.head(5)

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,ad_cost,ER%,influencer_size,avg_upload_interval,no.1,no.2,no.3
0,.blissdiaries,81700,9718,1447,60,196,194359333,1.850,마이크로 인플루언서,4.070,fashiion_tf,color_tf,skincare_tf
1,.m.egan,238200,178070,23864,183,3527,3561400000,10.100,메가 인플루언서,5.030,skincare_tf,color_tf,skin_routine_tf
2,.woniluv,135600,41956,2399,86,370,839124000,1.830,메가 인플루언서,2.380,skincare_tf,skin_routine_tf,trend_tf
3,4chriisty,79500,26122,4120,74,385,522450000,5.280,마이크로 인플루언서,2.480,fashiion_tf,color_tf,skincare_tf
4,__.annie.s,37300,10333,2375,128,465,206654667,6.710,마이크로 인플루언서,3.140,skincare_tf,asmr,fashiion_tf


In [46]:
mean_df[mean_df['name'].isin(["krystallee2222",'emchu_','velcroluv'])]

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,ad_cost,ER%,influencer_size,avg_upload_interval,no.1,no.2,no.3
18,emchu_,72700,39188,4090,34,549,783768667,5.670,마이크로 인플루언서,3.410,fashiion_tf,skincare_tf,skin_routine_tf
31,krystallee2222,65200,392343,35183,120,13671,7846866667,54.150,마이크로 인플루언서,2.240,skincare_tf,skin_routine_tf,color_tf
54,velcroluv,114500,21290,4664,222,922,425797333,4.270,메가 인플루언서,0.900,skincare_tf,skin_routine_tf,asmr


In [47]:
#얼굴 두분 다 나오고 

#krystallee2222 ER 높고 > 피부가 좋아서 신빙성 있음... 
#emchu_ 컨셉 



ver.1

In [48]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 인플루언서 데이터 로드 (예제)
final_df1 = mean_df.copy()

# 콘텐츠 컬럼 가중치 반영하여 합치기
final_df1['content_features'] = final_df1.apply(lambda x: 
    (x['no.1'] + ' ') * 3 + (x['no.2'] + ' ') * 2 + x['no.3'], axis=1)

# TF-IDF 변환
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(final_df1['content_features'])

# 기존 선택한 인플루언서 기준 유사도 측정
selected_influencers = ["krystallee2222", "emchu_"]
selected_indices = final_df1[final_df1['name'].isin(selected_influencers)].index
similarity_scores = cosine_similarity(tfidf_matrix, tfidf_matrix[selected_indices])

# 평균 유사도 점수 계산 후 정렬
final_df1["content_similarity"] = similarity_scores.mean(axis=1)
df_sorted = final_df1.sort_values(by="content_similarity", ascending=False)

# 최종 추천 인플루언서 리스트
top_n = 10  # 원하는 추천 개수
recommended_influencers = df_sorted.loc[~df_sorted['name'].isin(selected_influencers)].head(top_n)
print(recommended_influencers[["name", "content_similarity"]])

                name  content_similarity
14    cleangirlhacks               0.748
24        its.kaylas               0.666
13  caspertheghostyy               0.666
34    luvsfor.jennie               0.618
54         velcroluv               0.591
37        misscasxie               0.591
29            jeb_bi               0.591
40        mydelicate               0.582
9          asiamelon               0.582
55    widyaatharva01               0.573


ver.2

🔹 1. ER%를 가중치로 활용 (ER% 높은 인플루언서 선호)
no.1, no.2, no.3의 가중치를 ER%에 비례하도록 설정
ER%가 높은 인플루언서의 콘텐츠 키워드에 더 높은 점수를 부여


ER%가 높을수록 콘텐츠 키워드가 TF-IDF에 더 큰 영향을 주도록 조정
예를 들어, ER%가 **10.1%**인 인플루언서라면 no.1 키워드는 30.3배 반영
ER%가 **1.8%**인 인플루언서는 no.1이 5.4배 반영됨 → 자연스럽게 ER% 높은 인플루언서가 강조됨


ER%의 값이 너무 큰 경우(예: 1.8% vs. 10.1%) 가중치 차이가 심해질 수 있으므로 Min-Max Scaling을 적용하여 0~1 범위로 정규화하는 방법도 있음.



In [49]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

final_df2 = mean_df.copy()

scaler = MinMaxScaler()
final_df2['normalized_ER'] = scaler.fit_transform(final_df2[['ER%']])

# 콘텐츠 컬럼 가중치 반영하여 합치기
final_df2['content_features'] = final_df2.apply(lambda x: 
    (x['no.1'] + ' ') * 3 + (x['no.2'] + ' ') * 2 + x['no.3'], axis=1)

# TF-IDF 변환
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(final_df2['content_features'])

# 기존 선택한 인플루언서 기준 유사도 측정
selected_influencers = ["krystallee2222", "emchu_"]
selected_indices = final_df2[final_df2['name'].isin(selected_influencers)].index
similarity_scores = cosine_similarity(tfidf_matrix, tfidf_matrix[selected_indices])

# 평균 유사도 점수 계산 후 정렬
final_df2["content_similarity"] = similarity_scores.mean(axis=1)
df_sorted = final_df2.sort_values(by="content_similarity", ascending=False)

# 최종 추천 인플루언서 리스트
top_n = 10  # 원하는 추천 개수
recommended_influencers = df_sorted.loc[~df_sorted['name'].isin(selected_influencers)].head(top_n)
print(recommended_influencers[["name", "content_similarity"]])


final_df2['content_features'] = final_df2.apply(lambda x: 
    (x['no.1'] + ' ') * int(round(x['normalized_ER'] * 3, 0)) + 
    (x['no.2'] + ' ') * int(round(x['normalized_ER'] * 2, 0)) + 
    (x['no.3'] + ' ') * int(round(x['normalized_ER'], 0)),
    axis=1)

                name  content_similarity
14    cleangirlhacks               0.748
24        its.kaylas               0.666
13  caspertheghostyy               0.666
34    luvsfor.jennie               0.618
54         velcroluv               0.591
37        misscasxie               0.591
29            jeb_bi               0.591
40        mydelicate               0.582
9          asiamelon               0.582
55    widyaatharva01               0.573


# 콘텐츠 기반 필터링 추천 (Content based filtering) 

ver.3

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

final_df3 = mean_df.copy()

scaler = MinMaxScaler()
final_df3['normalized_ER'] = scaler.fit_transform(final_df3[['ER%']])

# 콘텐츠 컬럼 가중치 반영하여 합치기
final_df3['content_features'] = final_df3.apply(lambda x: 
    (x['no.1'] + ' ') * 3 + (x['no.2'] + ' ') * 2 + x['no.3'], axis=1)

NameError: name 'mean_df' is not defined

 벡터화(TF-IDF 변환)란? 왜 하는가?
TfidfVectorizer()를 사용하는 **벡터화(vectorization)**는 텍스트 데이터를 수치 데이터로 변환하는 과정입니다.
Neo4j처럼 그래프 구조를 쓰는 것이 아니라 머신러닝 모델을 활용하려면, 텍스트 데이터를 그대로 사용할 수 없기 때문에 숫자로 변환해야 합니다.

✅ TF-IDF (Term Frequency-Inverse Document Frequency) 벡터화

단어의 중요도를 반영하여 수치로 변환하는 방식
단순한 단어 등장 횟수(count) 기반 벡터화보다 중복 단어를 보정하여 의미 있는 특징을 추출
💡 왜 TF-IDF를 사용할까?

단순한 **단어 빈도수(CountVectorizer)**보다 의미 있는 정보 추출 가능
자주 등장하지만 중요하지 않은 단어(예: "the", "is")를 자동으로 낮은 가중치로 조정
가중치를 부여하여 인플루언서의 콘텐츠 특성을 더 잘 반영

In [2]:
# TF-IDF 변환
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(final_df3['content_features'])

NameError: name 'final_df3' is not defined

반영
🔹 유사도 측정 방식과 코사인 유사도 사용 이유
✅ 다양한 유사도 측정 방식

코사인 유사도 (Cosine Similarity)

두 벡터가 이루는 각도의 코사인 값을 이용해 유사도를 측정
값 범위: [-1, 1] (1에 가까울수록 유사, -1이면 정반대)
텍스트 데이터처럼 길이가 다를 수 있는 경우에도 적절한 유사도 측정 가능
유클리드 거리 (Euclidean Distance)

두 벡터 간의 직선 거리
텍스트 데이터처럼 길이 차이가 중요한 경우에는 부적절
자카드 유사도 (Jaccard Similarity)

두 문서에서 공통으로 등장하는 단어의 비율을 이용해 유사도 측정
단어 순서 정보가 반영되지 않음
점진적 유사도 (Dot Product Similarity)

단순한 내적(dot product) 방식
벡터 크기에 따라 결과가 영향을 받기 때문에 정규화되지 않은 데이터에서는 부적절
💡 왜 코사인 유사도를 선택했을까?

텍스트 벡터화 결과는 고차원 희소 벡터 형태가 됨
문서 간 길이(단어 개수) 차이가 크더라도 비교할 수 있도록 정규화된 방식이 필요
코사인 유사도는 벡터의 방향(특징 차이)만 비교하고, 크기는 무시하기 때문에 적절
즉, TF-IDF 변환된 콘텐츠 벡터들의 의미적 유사도를 비교하려면 코사인 유사도가 가장 적합한 선택입니다.

In [ ]:
# 기존 선택한 인플루언서 기준 유사도 측정
selected_influencers = ["krystallee2222", "emchu_"]
selected_indices = final_df3[final_df3['name'].isin(selected_influencers)].index
similarity_scores = cosine_similarity(tfidf_matrix, tfidf_matrix[selected_indices])

# 평균 유사도 점수 계산 후 정렬
final_df3["content_similarity"] = similarity_scores.mean(axis=1)
df_sorted = final_df3.sort_values(by="content_similarity", ascending=False)
df_sorted

In [ ]:
# 최종 추천 인플루언서 리스트
top_n = 10  # 원하는 추천 개수
recommended_influencers = df_sorted.loc[~df_sorted['name'].isin(selected_influencers)].head(top_n)
print(recommended_influencers[["name", "content_similarity"]])

# ER%가 너무 낮으면 가중치 값이 0이 되어 콘텐츠 키워드가 삭제될 위험 있음
# 이를 방지하려면 최소 1번은 반영되도록 max(1, int(...)) 적용
final_df3['content_features'] = final_df3.apply(lambda x: 
    (x['no.1'] + ' ') * max(1, int(round(x['normalized_ER'] * 3, 0))) + 
    (x['no.2'] + ' ') * max(1, int(round(x['normalized_ER'] * 2, 0))) + 
    (x['no.3'] + ' ') * max(1, int(round(x['normalized_ER'], 0))),
    axis=1) 



#(총 예상 투자 수익 – 투자 비용) / 투자 비용 x 100 = ROI%
#예를 들어, 인플루언서 마케팅 캠페인에 $2,000을 투자하고 $9,000의 가치를 얻을 것으로 예상한다면, ROI는 350%입니다.

#틱톡커의 과거 ROI 가 있다면 그에 따른 가중치 회귀분석과 그에 따른 예측이 가능하지 않을까???

 추천 접근법:

종속 변수(y): 광고 성과를 나타내는 지표 (예: sales, conversion_rate, ROI 등)
독립 변수(X): 광고 성과에 영향을 줄 가능성이 높은 지표
회귀 모델을 학습하여 가장 중요한 지표(가중치)를 찾고 최적의 인플루언서를 선정

In [103]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# 예제 데이터프레임
data = mean_df.copy()

#파생변수 생성,
data['like_ratio'] = data['like_cnt'] / data['view_cnt']
data['comment_ratio'] = data['comment_cnt'] / data['view_cnt']
data['save_ratio'] = data['save_cnt'] / data['view_cnt']
data['FSR'] = data['save_cnt'] / data['follower_cnt'] # Follower to Save Ratio (FSR)
data['EPV'] = (data['like_cnt'] + data['comment_cnt'] + data['save_cnt']) / data['view_cnt'] # 조회수 대비 얼마나 많은 참여(좋아요, 댓글, 저장)가 일어났는지를 나타내는 지표입니다


# 광고 비용 대비 반응률
data['response_rate'] = (data['like_cnt'] + data['comment_cnt'] + data['save_cnt']) / data['ad_cost']

data.head()

,name,follower_cnt,view_cnt,like_cnt,comment_cnt,save_cnt,ad_cost,ER%,influencer_size,avg_upload_interval,no.1,no.2,no.3,like_ratio,comment_ratio,save_ratio,FSR,EPV,response_rate
0,.blissdiaries,81700,9718,1447,60,196,194359333,1.850,마이크로 인플루언서,4.070,fashiion_tf,color_tf,skincare_tf,0.149,0.006,0.020,0.002,0.175,0.000
1,.m.egan,238200,178070,23864,183,3527,3561400000,10.100,메가 인플루언서,5.030,skincare_tf,color_tf,skin_routine_tf,0.134,0.001,0.020,0.015,0.155,0.000
2,.woniluv,135600,41956,2399,86,370,839124000,1.830,메가 인플루언서,2.380,skincare_tf,skin_routine_tf,trend_tf,0.057,0.002,0.009,0.003,0.068,0.000
3,4chriisty,79500,26122,4120,74,385,522450000,5.280,마이크로 인플루언서,2.480,fashiion_tf,color_tf,skincare_tf,0.158,0.003,0.015,0.005,0.175,0.000
4,__.annie.s,37300,10333,2375,128,465,206654667,6.710,마이크로 인플루언서,3.140,skincare_tf,asmr,fashiion_tf,0.230,0.012,0.045,0.012,0.287,0.000


# 회귀분석

In [ ]:
numeric_df = data.select_dtypes(include=['number'])
#상관계수 확인
numeric_df.corr()

In [97]:
# 다중공선성 위험 컬럼 제거 "ad_cost","view_cnt",'like_cnt','comment_cnt','save_cnt','response_rate,','FSR','EPV',
numeric_df = numeric_df.drop(columns=["ad_cost","view_cnt",'like_cnt','comment_cnt','save_cnt','response_rate','FSR','EPV'])
numeric_df.corr()

,follower_cnt,ER%,avg_upload_interval,like_ratio,comment_ratio,save_ratio
follower_cnt,1.000,-0.183,-0.179,-0.207,-0.244,-0.316
ER%,-0.183,1.000,0.035,0.102,-0.003,0.122
avg_upload_interval,-0.179,0.035,1.000,-0.119,-0.136,-0.159
like_ratio,-0.207,0.102,-0.119,1.000,0.773,0.846
comment_ratio,-0.244,-0.003,-0.136,0.773,1.000,0.790
save_ratio,-0.316,0.122,-0.159,0.846,0.790,1.000


In [98]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

result = add_constant(numeric_df)

# 각 변수에 대한 VIF 값 계산
vif_data = pd.DataFrame()
vif_data["feature"] = result.columns 
vif_data["VIF"] = [variance_inflation_factor(result.values, i) for i in range(result.shape[1])]

# VIF가 높은 변수 확인
print(vif_data)

               feature   VIF
0                const 7.978
1         follower_cnt 1.233
2                  ER% 1.074
3  avg_upload_interval 1.096
4           like_ratio 4.028
5        comment_ratio 3.068
6           save_ratio 4.583


In [99]:
from sklearn.linear_model import LinearRegression
# 독립 변수와 종속 변수 준비
X = numeric_df[['like_ratio', 'comment_ratio', 'save_ratio', 'follower_cnt', 'avg_upload_interval']]
X.corr()

,like_ratio,comment_ratio,save_ratio,follower_cnt,avg_upload_interval
like_ratio,1.000,0.773,0.846,-0.207,-0.119
comment_ratio,0.773,1.000,0.790,-0.244,-0.136
save_ratio,0.846,0.790,1.000,-0.316,-0.159
follower_cnt,-0.207,-0.244,-0.316,1.000,-0.179
avg_upload_interval,-0.119,-0.136,-0.159,-0.179,1.000


In [113]:
y = numeric_df['ER%'] 

# 학습 / 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 회귀 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("회귀 계수 (가중치):", dict(zip(X.columns, model.coef_.round(2))))
print(f"MAE: {mae:.5f}, R²: {r2:.5f}")

회귀 계수 (가중치): {'like_ratio': 77.42, 'comment_ratio': -1456.82, 'save_ratio': 269.25, 'follower_cnt': -0.0, 'avg_upload_interval': -0.06}
MAE: 11.85758, R²: -0.53291
